## 使用Python实现Decision Tree算法

### 1、基尼系数

基尼系数是用于评估数据集中分割的成本函数的名称。

In [1]:
def gini_index(groups, class_values):
    gini = 0.0
    for class_value in class_values:
        for group in groups:
            size = len(group)
            if size == 0:
                continue
            proportion = [row[-1] for row in group].count(class_value) / float(size)
            gini += (proportion * (1.0 - proportion))
    return gini

我们可以设计一个小数据集来测试我们的gini_index()函数。

In [2]:
print ((gini_index([[[1, 1], [1, 0]], [[1, 1], [1, 0]]], [0, 1])))
print ((gini_index([[[1, 0], [1, 0]], [[1, 1], [1, 1]]], [0, 1])))

1.0
0.0


### 2、创建划分

#### 2.1、划分一个数据集

拆分数据集意味着在给定属性的索引和该属性的拆分值的情况下将数据集分为两组。

一旦我们有两组，我们可以使用我们的Gini得分来评估划分的情况。

划分数据集包括对每行进行迭代，检查属性值是低于还是高于划分值，并分别将其分配给左侧组或右侧组。

In [3]:
def test_split(index, value, dataset):
    left, right = [], []
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right

#### 2.2、评估全部划分

使用上面的Gini函数和测试划分函数，我们现在来评估拆分。

给定一个数据集，我们必须检查每个属性上的每个值作为候选划分，评估划分的效果，并找到我们可能做出的最佳划分。

当最好的划分确定之后，我们可以用来作为决策树的节点。

我们将使用字典来表示决策树中的一个节点，因为我们可以按名称存储数据。 当选择最佳划分并将其用作树的新节点时，我们将存储所选属性的索引，要划分的属性的值以及由所选划分点划分的两组数据。

In [4]:
def get_split(dataset):
    class_values = list(set(row[-1] for row in dataset))
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    for index in range(len(dataset[0])-1):
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            gini = gini_index(groups, class_values)
            print('X%d < %.3f Gini=%.3f' % ((index+1), row[index], gini))
            if gini < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups
    return {'index':b_index, 'value':b_value, 'groups':b_groups}

我们可以设计一个小数据集来测试我们的get_split()函数。

In [5]:
dataset = [[2.771244718,1.784783929,0],
    [1.728571309,1.169761413,0],
    [3.678319846,2.81281357,0],
    [3.961043357,2.61995032,0],
    [2.999208922,2.209014212,0],
    [7.497545867,3.162953546,1],
    [9.00220326,3.339047188,1],
    [7.444542326,0.476683375,1],
    [10.12493903,3.234550982,1],
    [6.642287351,3.319983761,1]]
split = get_split(dataset)
print (('Split: [X%d < %.3f]' % ((split['index']+1), split['value'])))

X1 < 2.771 Gini=0.494
X1 < 1.729 Gini=0.500
X1 < 3.678 Gini=0.408
X1 < 3.961 Gini=0.278
X1 < 2.999 Gini=0.469
X1 < 7.498 Gini=0.408
X1 < 9.002 Gini=0.469
X1 < 7.445 Gini=0.278
X1 < 10.125 Gini=0.494
X1 < 6.642 Gini=0.000
X2 < 1.785 Gini=1.000
X2 < 1.170 Gini=0.494
X2 < 2.813 Gini=0.640
X2 < 2.620 Gini=0.819
X2 < 2.209 Gini=0.934
X2 < 3.163 Gini=0.278
X2 < 3.339 Gini=0.494
X2 < 0.477 Gini=0.500
X2 < 3.235 Gini=0.408
X2 < 3.320 Gini=0.469
Split: [X1 < 6.642]


### 3、建立一颗决策树

#### 3.1、终止节点

我们需要决定什么时候停止树的成长。我们可以使用节点在训练数据集中到达的深度和行数来做到这一点。

最大树深度：这是从树的根节点开始的最大节点数。一旦满足树的最大深度，我们必须停止分裂添加新节点。更深的树更复杂，更可能过拟合训练数据。

最小节点数：这是给定节点负责的训练模式的最小数量。 一旦达到或低于此最小值，我们必须停止拆分和添加新节点。

In [6]:
def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)

#### 3.2、递归划分

In [7]:
def split(node, max_depth, min_size, depth):
    left, right = node['groups']
    del(node['groups'])
    # 不再划分
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    # 最大深度
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    # 左子树
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left)
        split(node['left'], max_depth, min_size, depth+1)
    # 右子树
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right)
        split(node['right'], max_depth, min_size, depth+1)

#### 3.3、创建决策树

In [8]:
def build_tree(train, max_depth, min_size):
    root = get_split(dataset)
    split(root, max_depth, min_size, 1)
    return root

这里再增加一个print_tree()函数，它递归地打印出决策树的节点，每个节点一行。 虽然不如真正的决策树图那么直观，但是它给出了决策树的基本生成思路。

In [9]:
def print_tree(node, depth=0):
    if isinstance(node, dict):
        print ('%s[X%d < %.3f]' % ((depth*' ', (node['index']+1), node['value'])))
        print_tree(node['left'], depth+1)
        print_tree(node['right'], depth+1)
    else:
        print ('%s[%s]' % ((depth*' ', node)))

我们可以设计一个小数据集来查看我们的决策树是如何生成的。

In [10]:
dataset = [[2.771244718,1.784783929,0],
    [1.728571309,1.169761413,0],
    [3.678319846,2.81281357,0],
    [3.961043357,2.61995032,0],
    [2.999208922,2.209014212,0],
    [7.497545867,3.162953546,1],
    [9.00220326,3.339047188,1],
    [7.444542326,0.476683375,1],
    [10.12493903,3.234550982,1],
    [6.642287351,3.319983761,1]]
tree = build_tree(dataset, 1, 1)
print_tree(tree)

X1 < 2.771 Gini=0.494
X1 < 1.729 Gini=0.500
X1 < 3.678 Gini=0.408
X1 < 3.961 Gini=0.278
X1 < 2.999 Gini=0.469
X1 < 7.498 Gini=0.408
X1 < 9.002 Gini=0.469
X1 < 7.445 Gini=0.278
X1 < 10.125 Gini=0.494
X1 < 6.642 Gini=0.000
X2 < 1.785 Gini=1.000
X2 < 1.170 Gini=0.494
X2 < 2.813 Gini=0.640
X2 < 2.620 Gini=0.819
X2 < 2.209 Gini=0.934
X2 < 3.163 Gini=0.278
X2 < 3.339 Gini=0.494
X2 < 0.477 Gini=0.500
X2 < 3.235 Gini=0.408
X2 < 3.320 Gini=0.469
[X1 < 6.642]
 [0]
 [1]


### 4、作出预测

In [11]:
def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']

我们可以设计一个小数据集来使用决策树预测。

In [12]:
dataset = [[2.771244718,1.784783929,0],
    [1.728571309,1.169761413,0],
    [3.678319846,2.81281357,0],
    [3.961043357,2.61995032,0],
    [2.999208922,2.209014212,0],
    [7.497545867,3.162953546,1],
    [9.00220326,3.339047188,1],
    [7.444542326,0.476683375,1],
    [10.12493903,3.234550982,1],
    [6.642287351,3.319983761,1]]
 
stump = {'index': 0, 'right': 1, 'value': 6.642287351, 'left': 0}
for row in dataset:
    prediction = predict(stump, row)
    print ('Expected=%d, Got=%d' % (row[-1], prediction))

Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
